In [2]:
%load_ext autoreload
%autoreload 2

In [9]:
import us
import copy
import tqdm.auto as tqdm
import geopandas as gpd
import numpy as np
import pandas as pd
from shapefiles import shapefiles, school_district_shapefiles, ce_to_name
from relationship import full_relationships, create_relationships
from stats_for_shapefile import compute_statistics_for_shapefile, compute_summed_shapefile_all_keys, sum_keys
from create_website import (
    full_shapefile,
    shapefile_without_ordinals,
    add_ordinals,
    get_statistic_categories,
    create_page_jsons,
)
from election_data import with_election_results
from urbanstats.features.extract_data import feature_data
import matplotlib.pyplot as plt
import matplotlib as mpl

from urbanstats.data.gpw import *
from urbanstats.data.population_urban_area import *

from collections import defaultdict, Counter


from shapefiles import CONGRESSIONAL_DISTRICTS, COUNTIES, county_cross_cd, usda_county_type, shapefiles

In [10]:
from urbanstats.osm.buses import national_stops
from urbanstats.osm.trains import national_stations

In [70]:
remap_names = {
        "cats": "charlotte area transit system",
    "bnsf railway": "bnsf",
    "detroit people mover co.": "detroit people mover co"

}
for x in {
}:
    remap_names[x] = "utah transit authority"

In [84]:
import re

def canonicalize(name):
    # Convert to lowercase and remove leading/trailing whitespaces
    name = name.lower().strip()

    # Remove special characters and replace spaces with underscores
    name = re.sub(r'[^a-z0-9\s]', '', name)
    name = re.sub(r'\s+', '_', name)

    # Handle specific cases
    if 'metropolitan transportation authority' in name:
        return 'mta'
    elif 'long island rail road' in name or 'long island railroad' in name:
        return 'lirr'
    elif 'san francisco bay area rapid transit district' in name:
        return 'bart'
    elif 'massachusetts bay transportation authority' in name:
        return 'mbta'
    elif 'new york city transit authority' in name:
        return 'nycta'
    elif 'niagara frontier transportation authority' in name:
        return 'nfta'
    elif 'new hope & ivyland rail road' in name:
        return 'new_hope_ivyland_rr'
    elif 'bi-state development agency' in name:
        return 'bi_state_development_agency'
    elif 'chicago transit authority' in name:
        return 'cta'
    elif 'peninsula corridor joint powers board' in name:
        return 'caltrain'
    elif 'port authority of new york and new jersey' in name:
        return 'port_authority'
    elif 'new jersey transit' in name:
        return 'nj_transit'
    elif 'southeastern pennsylvania transportation authority' in name:
        return 'septa'
    elif 'washington metropolitan area transit authority' in name:
        return 'wmata'
    elif 'los angeles county transportation authority' in name:
        return 'lacmta'
    elif 'santa clara valley transportation authority' in name:
        return 'vta'
    else:
        # If no specific rule matches, return the cleaned name
        return name

In [106]:
stations

,name,tags,geometry,network,operator,exclusion_reason
0,1st Street,"{'light_rail': 'yes', 'name': '1st Street', 'n...",POINT (-118.18940 33.76871),LACMTA,los angeles county metropolitan transportation...,
1,Downtown Long Beach,"{'alt_name': 'Transit Mall', 'light_rail': 'ye...",POINT (-118.19296 33.76804),LACMTA,los angeles county metropolitan transportation...,
2,5th Street,"{'light_rail': 'yes', 'name': '5th Street', 'n...",POINT (-118.18938 33.77354),LACMTA,los angeles county metropolitan transportation...,
3,Downers Grove Main Street,"{'addr:city': 'Downers Grove', 'addr:housenumb...",POINT (-88.00863 41.79554),Metra,metra,
4,Stone Avenue,"{'addr:city': 'La Grange', 'addr:housenumber':...",POINT (-87.87792 41.81433),Metra,metra,
...,...,...,...,...,...,...
4327,Metrolink East Ontario Station,"{'name': 'Metrolink East Ontario Station', 'ne...",GEOMETRYCOLLECTION EMPTY,Metrolink,southern california regional rail authority,
4328,Nairobi,"{'name': 'Nairobi', 'public_transport': 'stop_...",GEOMETRYCOLLECTION EMPTY,NO NETWORK,no operator,
4329,Stanleyville,"{'name': 'Stanleyville', 'public_transport': '...",GEOMETRYCOLLECTION EMPTY,NO NETWORK,no operator,
4330,Marietta Station,"{'historic': 'railway', 'name': 'Marietta Stat...","MULTIPOLYGON (((-84.55098 33.95414, -84.55098 ...",NO NETWORK,no operator,


In [144]:
stations[stations.name.apply(lambda x: "Ontario" in x)]

,name,tags,geometry,network,operator,exclusion_reason
2378,Ontario,"{'addr:state': 'CA', 'name': 'Ontario', 'netwo...",POINT (-117.64959 34.06163),Amtrak,amtrak,Amtrak
2421,East Ontario,"{'name': 'East Ontario', 'network': 'Metrolink...",POINT (-117.57972 34.03999),Metrolink,southern california regional rail authority,
4327,Metrolink East Ontario Station,"{'name': 'Metrolink East Ontario Station', 'ne...",GEOMETRYCOLLECTION EMPTY,Metrolink,southern california regional rail authority,


In [143]:
list(stations[stations.name.apply(lambda x: "Ontario" in x)].tags)

[{'addr:state': 'CA',
  'name': 'Ontario',
  'network': 'Amtrak',
  'operator': 'Amtrak',
  'public_transport': 'station',
  'railway': 'station',
  'railway:ref': 'ONA',
  'train': 'yes',
  'website': 'https://www.amtrak.com/stations/ona.html',
  'wikidata': 'Q7094586',
  'wikipedia': 'en:Ontario station (Amtrak)'},
 {'name': 'East Ontario',
  'network': 'Metrolink',
  'network:wikidata': 'Q766647',
  'operator': 'Southern California Regional Rail Authority',
  'public_transport': 'station',
  'railway': 'station',
  'railway:position': 'mi:42.2',
  'train': 'yes',
  'wikidata': 'Q5329075',
  'wikipedia': 'en:East Ontario (Metrolink station)'},
 {'name': 'Metrolink East Ontario Station',
  'network': 'Metrolink',
  'network:wikidata': 'Q766647',
  'operator': 'Southern California Regional Rail Authority',
  'public_transport': 'station',
  'railway': 'station',
  'source': 'survey;image;yahoo_imagry',
  'source_ref': 'AM909_DSCN2230',
  'train': 'yes',
  'type': 'site'}]

In [135]:
stations = national_stations().copy()
get_non_excluded = lambda: stations[stations.exclusion_reason == ""]
stations["exclusion_reason"] = ""
stations.loc[stations["operator"] == "amtrak", "exclusion_reason"] = "Amtrak"
stations.loc[stations["operator"] == "bnsf", "exclusion_reason"] = "BNSF"
stations.loc[stations["operator"] == "NO OPERATOR", "exclusion_reason"] = "No operator"
# stations.operator = stations.operator.apply(lambda x: canonicalize(x))
get_operator_counts = lambda: Counter(
    x for xs in get_non_excluded().operator for x in xs.split(";")
)
# operator_counts = get_operator_counts()
too_small_operators = {x for x in operator_counts if operator_counts[x] <= 5}
# stations.loc[
#     stations["operator"].apply(
#         lambda operators: all(
#             operator in too_small_operators for operator in operators.split(";")
#         )
#     ),
#     "exclusion_reason",
# ] = "Operator too small"
# non_excluded = get_non_excluded()
operator_counts = get_operator_counts()

In [136]:
[x for x in stations["operator"] if ";" in x or "&" in x or " and " in x or ',' in x]

[]

In [138]:
print("\n".join(repr(x) for x in sorted(operator_counts)))
# len(operator_counts)

'adirondak scenic railroad'
'adrian_and_blissfield_railroad'
'alaska railroad'
'alaska railroad corporation'
'altamont corridor express'
'architect of the capitol'
'b_and_o'
'bart'
'bellefonte historical railroad society'
'bi_state_development_agency'
'big moose station restaraunt'
'black hills central railroad'
'black_river_and_western'
'blue ridge scenic railway'
'boone_and_scenic_valley_railroad'
'branson scenic railway'
'brightline'
'busch gardens williamsburg'
'california state railroad museum'
'caltrain'
'caltrans'
'camp noyo'
'capital metro'
'capital metropolitan transportation authority'
'catskill mounrain railroad'
'ccrta'
'cedar park austin steam train association'
'charlotte_area_transit_system'
'chattanooga society of model engineers'
'chehalis_centralia_railroad_and_museum'
'city of albuquerque'
'city of barstow'
'city of palmdale'
'city of tustin'
'city_and_county_of_honolulu'
"clark's trading post"
'closed'
'colebrookdale railroad'
'connecticut department of transportati

In [127]:
set(stations[stations.operator == "amtrak"].exclusion_reason)

{'Amtrak'}

In [128]:
operator_counts

Counter({'lacmta': 102,
         'metra': 196,
         'no_operator': 1416,
         'caltrans': 1,
         'nycta': 445,
         'mta': 50,
         'lirr': 80,
         'bart': 49,
         'mbta': 143,
         'nfta': 13,
         'new_hope_ivyland_rr': 1,
         'septa': 148,
         'bi_state_development_agency': 38,
         'cta': 140,
         'caltrain': 2,
         'port_authority': 4,
         'nj_transit': 72,
         'path': 9,
         'sfmuni': 58,
         'ohare_ats': 5,
         'disney_transport': 2,
         'marta': 3,
         'patco': 4,
         'detroit people mover co.': 9,
         'metro-north railroad': 71,
         'utah transit authority': 22,
         'maryland transit administration': 58,
         'csx': 4,
         'south florida regional transportation authority': 16,
         'north county transit district': 5,
         'regional transportation district': 58,
         'washington metropolitan area transit authority': 102,
         'miami-dade

In [129]:
"; ".join(sorted(too_small_operators))

"adirondak scenic railroad; adrian and blissfield railroad; alaska railroad; alaska railroad corporation; altamont corridor express; bellefonte historical railroad society; big moose station restaraunt; black hills central railroad; black river & western rr; black river and western; blue ridge scenic railway; boone_and_scenic_valley_railroad; branson scenic railway; brightline; busch gardens williamsburg; california state railroad museum; caltrain; caltrans; camp noyo; capital metro; catskill mounrain railroad; ccrta; cedar park austin steam train association; charlotte area transit system; chattanooga society of model engineers; chehalis centralia railroad & museum; city and county of honolulu; city of albuquerque; city of barstow; city of palmdale; city of tustin; clark's trading post; closed; colebrookdale railroad; connecticut department of transportation; conway scenic railroad; cooperstown & charlotte valley railroad; coumbia sussex; csx; csx transportation; ctrail; cumbres and t

In [92]:
systems = pd.DataFrame(
    {k: {"count": v} for k, v in operator_counts.items()}
).T.sort_values("count")

In [93]:
systems.to_csv("/home/kavi/temp/temp.csv")

In [94]:
systems[systems["count"] >= 10]

,count
uta,10
detroit_people_mover_co,11
nfta,12
sonomamarin_area_rail_transit_district,12
south_florida_regional_transportation_authority,12
sunrail,14
denver_transit_partners,15
port_authority_of_allegheny_county,16
lirr,19
long_island_railroad,19


In [34]:
def save(fr, path):
    fr = fr.copy()
    print(list(fr.tags))
    fr["tags"] = fr["tags"].apply(str)
    fr.geometry = fr.geometry.centroid
    fr.to_file(f"/home/kavi/temp/temp_shapefiles/{path}.shp")

In [27]:
save(stations[stations.operator.apply(lambda os: "ripta" in os.split(";"))], "ripta")

[{'addr:city': 'Pawtucket', 'addr:housenumber': '300', 'addr:postcode': '02860', 'addr:state': 'RI', 'addr:street': 'Pine Street', 'area': 'yes', 'check_date': '2023-01-23', 'name': 'Pawtucket/Central Falls', 'network': 'MBTA;RIPTA', 'opening_date': '2023-01-23', 'operator': 'Keolis;RIPTA', 'public_transport': 'station', 'railway': 'station', 'railway:position': 'mi:189.3', 'train': 'yes', 'website': 'https://www.mbta.com/', 'wikidata': 'Q107322761', 'wikipedia': 'en:Pawtucket/Central Falls station'}]


/home/kavi/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  


DriverIOError: Failed to create file /home/kavi/temp/temp_shapefiles/ripta.shp: No such file or directory

In [ ]:
stations.to_csv("/home/kavi/temp/temp_shapefiles/temp.csv")

In [ ]:
list(stations[stations.network == "Baltimore Light RailLink"].tags)

In [ ]:
stations

In [ ]:
bad_networks = network_counts.name <= 5
good_networks = network_counts.name >= 50

In [ ]:
network_counts[(~bad_networks) & (~good_networks)].sort_values("name")

In [ ]:
"; ".join(sorted([x for x in counts if counts[x] <= 5]))

In [ ]:
save(non_excluded, "non_excluded_1")

In [ ]:
import us

In [ ]:
# help(overpy.Overpass)

In [ ]:
import shapely
import tqdm.auto as tqdm

In [ ]:
import shapely

In [ ]:
list(non_excluded[non_excluded.network == "NO NETWORK"].tags)

In [ ]:
# x = (
#     gpd.GeoDataFrame(geometry=[sh], crs="epsg:4326")
#     .to_crs(crs={"proj": "cea"})
#     .explode(index_parts=True)
#     .reset_index(drop=True)
# )

In [ ]:
r = 1

In [ ]:
chunk_size = 10_000

In [ ]:
pd.concat(results)

In [ ]:
%time x2 = parks_exploded()

In [ ]:
x2

In [ ]:
r = 1
%time _ = .ipynb_checkpoints/(sh)

In [ ]:
type(sh)

In [ ]:
sh.geoms[4]

In [ ]:
geom = result[result.name == "Lowery Park"].iloc[-1].geometry

In [ ]:
geom.bounds

In [ ]:
# 30m accuracy = 0.0003 degrees
precision = 0.0003
coords = np.array(geom.boundary.segmentize(precision).coords)
((coords[1:] - coords[:-1]) ** 2).sum(-1) ** 0.5

In [ ]:
%time len(pickle.dumps(result)) / 1e9

In [ ]:
import shapely

In [ ]:
result.geometry.apply(
    lambda x: x.buffer(1e-3) if isinstance(x, shapely.Point) else x
).to_file("/home/kavi/temp/temp.shp")

In [ ]:
polygon_for_way(ways_with_tags[1])

In [ ]:
%time out = polygon_for_relation(relation)

In [ ]:
out

In [ ]:
members["outer"][1]

In [ ]:
[x for x in relation.members if x.role == "outer"][5].resolve()

In [ ]:
members["outer"]

In [ ]:
unary_union(members["outer"][:1000])

In [ ]:
way.resolve().__dict__

In [ ]:
result.ways

In [ ]:
relation.__dict__

In [ ]:
polygon_for_way(result.ways[2])

In [ ]:
result.ways[0].nodes

In [ ]:
rj = json.loads(result.decode("utf-8"))

In [ ]:
def convert_json(items):
    import json
    return json.dumps({ "type": "FeatureCollection",
                        "features": [ 
                                        {"type": "Feature",
                                         "geometry": { "type": "Point",
                                                       "coordinates": [ feature['lon'],
                                                                        feature['lat']]},
                                         "properties": { key: value 
                                                         for key, value in feature.items()
                                                         if key not in ('lat', 'lon') }
                                         } 
                                     for feature in json.loads(items)
                                    ]
                       })


In [ ]:
result = json.loads(result.decode("utf-8"))

In [ ]:
convert_json(json.dumps(result["elements"]))

In [ ]:
with open("/home/kavi/temp/temp.geojson", "wb") as f:
    f.write(result)

In [ ]:
result.relations[0].members

In [ ]:
import overpass

In [ ]:
api = overpass.API()


In [ ]:
result = api.get(
q
)

with open("/home/kavi/temp/temp.geojson", "w") as f:
    json.dump(result, f)

In [ ]:
gpd.read_file("/home/kavi/temp/temp.geojson")

In [ ]:
overpass.to

In [ ]:
result = full_shapefile()

In [ ]:
from urbanstats.export.export_csvs import export_all_csvs

In [ ]:
result.shape[0]

In [ ]:
export_all_csvs("/home/kavi/temp/exported-urbanstats/csvs/")

In [ ]:
category = "main"

In [ ]:
csv_for(type, category)

In [ ]:
.to_csv("/home/kavi/temp/temp.csv")

# ! rm ~/temp/temp.csv.gz
# ! gzip ~/temp/temp.csv
# ! du -sh ~/temp/temp.csv.gz

In [ ]:
result.groupby("type").count().longname.sort_values()

In [ ]:
from census_blocks import load_raw_census

In [ ]:
# import osmium
# import tqdm.auto as tqdm

# class NamesHandler(osmium.SimpleHandler):
#     def __init__(self):
#         super().__init__()
#         self.pbar = tqdm.tqdm()
#     def node(self, n):
#         self.pbar.update()
#         if 'leisure' in n.tags and n.tags["leisure"] == "park":
#             print(n)
# #         if len(self.nodes):
# #             1/0


In [ ]:
# NamesHandler().apply_file("named_region_shapefiles/features/us-latest.osm.pbf")

In [ ]:
_, pop, _, _, coordinates = load_raw_census()

In [ ]:
census_blocks = gpd.GeoDataFrame(
    geometry=gpd.points_from_xy(coordinates[:, 1], coordinates[:, 0], crs="epsg:4326")
)

In [ ]:
from urbanstats.features.feature import features, shapefile_points_to_radius
from urbanstats.features.within_distance import features_within_distance_by_block, minimum_distance_by_block
from urbanstats.features.extract_data import feature_data


In [ ]:
feature_data()

In [ ]:
pop.shape

In [ ]:
feature = features["hospitals"]

In [ ]:
minimum_distance_by_block(feature)

In [ ]:
import geopy

In [ ]:
from geopy.distance import geodesic


In [ ]:
by_block = features_within_distance_by_block(feature)

In [ ]:
pop[by_block != 0].sum() / pop.sum()

In [ ]:
plt.figure(dpi=2000, figsize=(10, 10))
y, x = coordinates[::100].T
d = distance[::100]
mask = (y > 30) & (y < 40) & (x > -125) & (x < -115)
y, x, d = y[mask], x[mask], d[mask]
d = np.clip(d, 0, 100)
plt.scatter(x, y, c=d, marker=".", s=0.1)
plt.axis("equal")

In [ ]:
plt.figure(dpi=200, figsize=(10, 10))
def plot(cs, **kwargs):
    y, x = cs[::10].T
    mask = (y > 25) & (y < 40) & (x > -107) & (x < -92)
    y, x = y[mask], x[mask]
    plt.scatter(x, y, marker=".", s=0.1, **kwargs)
plot(coordinates[by_block != 0])
plot(coordinates[by_block == 0])
plt.axis("equal")

In [ ]:
ell.lon_radius, ell.lat_radius

In [ ]:
shapely.affinity.scale

In [ ]:
shape.bounds

In [ ]:
(-95.04905911509323) - (-94.84189452490675)

In [ ]:
29.657688059408127 - 29.837552380591873

In [ ]:
from urbanstats.acs.attach import with_acs_data, combine_us_pr
from urbanstats.acs.entities import entities, entities_split_by_usa_pr
from urbanstats.acs.load import get_acs_data, acs_variables, ACSDataEntity
ent = entities["ancestry"]
# ent= entities_split_by_usa_pr["birthplace"]
# get_acs_data(ent).sum() / get_acs_data(ent).sum().sum()
# combine_us_pr(*ent).sum() / combine_us_pr(*ent).sum().sum()

In [ ]:
table_eurafrica = get_acs_data.function(entities["ancestry"]).sum()

In [ ]:
table_asian = get_acs_data.function(entities["ancestry_asian"]).sum()

In [ ]:
table_hispanic = get_acs_data.function(entities["ancestry_hispanic"]).sum()

In [ ]:
amount = pd.DataFrame([table_asian, table_eurafrica, table_hispanic]).sum().sort_values()

In [ ]:
amount.sum() / 1e6

In [ ]:
for x in amount.index:
    print(f"{x:30s}{amount[x] / 1e6:6.2f}")

In [ ]:
amount.shape

In [ ]:
len(sum_keys)

In [ ]:
a = compute_statistics_for_shapefile(shapefiles["states"])
a[["shortname", "year_built_2010_or_later"]]

In [ ]:
folder = '/home/kavi/temp/site/'

In [ ]:
create_page_json(
    f"{folder}/data",
    u,
    relationships,
    long_to_short,
    long_to_pop,
    long_to_type,
)

In [ ]:
a = data_files_pb2.Article()

In [ ]:
from shapefiles import shapefiles
from output_geometry import produce_all_geometry_json

In [ ]:
x = shapefiles["cities"].load_file()

In [ ]:
import geopandas as gpd

In [ ]:
y = gpd.read_file("/home/kavi/Downloads/cb_2022_us_place_500k.zip")

In [ ]:
x.iloc[-200]

In [ ]:
produce_all_geometry_json("/home/kavi/temp/site/shape/", [])

In [ ]:
with open("/home/kavi/temp/site/shape/Fairfax County, Virginia, USA.json") as f:
    fc_python = json.load(f)

In [ ]:
from urbanstats.protobuf import data_files_pb2

In [ ]:
from output_geometry import convert_to_protobuf

In [ ]:
fc = convert_to_protobuf(fc_python)

In [ ]:
import gzip

In [ ]:
with gzip.open(f"/home/kavi/temp/site/shape/Fairfax County, Virginia, USA.gz", "wb") as f:
    f.write(fc.SerializeToString())

In [ ]:
fc_2 = data_files_pb2.FeatureCollection()
with gzip.open("/home/kavi/temp/site/shape/Fairfax County, Virginia, USA.gz", "rb") a sf:
    fc_2.

In [ ]:
! du -sb ~/temp.gz

In [ ]:
len(json.dumps(fc_python))

In [ ]:
ring_python

In [ ]:
f.

In [ ]:
x[0]["coordinates"]

In [ ]:
for i in tqdm.trange(full.shape[0], desc="creating pages"):
    row = full.iloc[i]
#     if row["type"] != "State":
#         continue
    create_page_json(
        f"{folder}/data",
        row,
        relationships,
        long_to_short,
        long_to_pop,
        long_to_type,
    )


In [ ]:
# get_acs_data(entities["aggregate_rent"]).sum()

In [ ]:
sf = shapefiles["states"]

In [ ]:
x = sf.load_file().to_crs({"proj": "cea"})

In [ ]:
area = x.geometry.area
perimiter = x.geometry.length
x["compactness"] = 4 * np.pi * area / perimiter ** 2

In [ ]:
x[["shortname", "compactness"]].sort_values("compactness")

In [ ]:
x.sort_values("education_field_humanities")

In [ ]:
x.education_field_humanities

In [ ]:
y = with_acs_data()

In [ ]:
y[y.geoid.apply(lambda x: "US28" in x)].education_field_humanities.sum()

In [ ]:
import pickle
len(pickle.dumps(with_acs_data()))

In [ ]:
sorted(
    set(
        [
            v["concept"]
            for v in acs_variables().values()
            if "$10,000" in v["label"]
            if "HOUSEHOLD" not in v["concept"]
            and "FAMILY" not in v["concept"]
            and "INC" in v["concept"]
        ]
    )
)

In [ ]:
var_for_concept = for_concept(
    
)

In [ ]:
data = with_election_results()

In [ ]:
var_for_concept

In [ ]:
universe_for_disagg = "population_18"

In [ ]:
%time 

In [ ]:
acs_data

In [ ]:
data[universe_for_disagg]

In [ ]:
result.state + result.county + result.tract

In [ ]:
list(x)

In [ ]:
key = ("2020 Presidential Election", "margin")

In [ ]:
y = shapefile_without_ordinals()

In [ ]:
y_n = add_ordinals(y[y.type == "Neighborhood"])

In [ ]:
y_n.loc[
    [
        1564,
        3907,
        302,
        4009,
        3464,
        1712,
        3166,
        3457,
        3568,
        5871,
        5886,
        1871,
        3757,
        3454,
        1635,
        1337,
        3027,
        2704,
        406,
        5623,
        4024,
        3906,
        1429,
        6049,
        108,
        643,
        612,
        596,
        599,
        5477,
        1924,
        3408,
        3169,
        2273,
        2164,
        598,
        5822,
        1679,
        3500,
        3527,
        2414,
        4397,
        4312,
        4106,
        6055,
        5466,
        2363,
        1804,
        1723,
        4027,
        1591,
        1764,
        4627,
        3465,
        3800,
        3663,
        1376,
        1705,
        5395,
        98,
        2064,
        2065,
        2848,
        1916,
        6129,
        1610,
        3967,
        2409,
        3575,
        396,
        3963,
        1834,
        1664,
        1614,
        5805,
        1543,
        3997,
        5015,
        3998,
        2293,
        3565,
        2682,
        2413,
        3000,
        2511,
        1661,
        1620,
        5614,
        2415,
        2869,
        3146,
        1631,
        1806,
        3026,
        2463,
        5641,
        6031,
        1791,
        1879,
        4507,
        404,
        1450,
        1381,
        1740,
        2747,
        4127,
        1346,
        4502,
        79,
        1417,
        1462,
        1460,
        1681,
        2042,
        1745,
        1897,
        1981,
        1983,
        3479,
        3528,
        5449,
        5450,
        5447,
        4538,
        1410,
        5710,
        398,
        1741,
        1926,
        4109,
        1847,
        4047,
        3452,
        3210,
        2555,
        3566,
        4869,
        5271,
        5115,
        6107,
        3165,
        2205,
        5467,
        1259,
        4247,
        1785,
        4048,
        4734,
        3917,
        1807,
        3458,
        679,
        3901,
        2400,
        5265,
        3376,
        3022,
        2082,
        5637,
        2148,
        301,
        5476,
        1633,
        5456,
        2683,
        6087,
        3333,
        2833,
        4981,
        310,
        2637,
        5680,
        5693,
        5712,
        1997,
        3504,
        4691,
        3661,
        3571,
        5707,
        5475,
        4919,
        3047,
        3306,
        2037,
        4294,
        1387,
        6027,
        3764,
        2436,
        1683,
        5458,
        5470,
        5685,
        6025,
        3383,
        3572,
        1853,
        2298,
        4530,
        6083,
        3921,
        1312,
        2286,
        6137,
        5713,
        3797,
        5000,
        5287,
        5499,
        3956,
        2302,
        2701,
        5282,
        5684,
        2304,
        3115,
        749,
        3410,
        3942,
        5698,
        6085,
        6118,
        1547,
        2743,
        4030,
        2253,
        1227,
        1844,
        5280,
        3471,
        3590,
        5754,
        3407,
        1972,
        112,
        177,
        180,
        2340,
        3384,
        3330,
        3925,
        5700,
        3662,
        3937,
        5596,
        1743,
        1527,
        4685,
        4391,
        4230,
        6088,
        5806,
        1593,
        2866,
        2076,
        3736,
        4570,
        5816,
        6059,
        3750,
        1232,
        1599,
        2377,
        1291,
        2720,
        1715,
        3953,
        237,
        220,
        2195,
        4677,
        4286,
        5812,
        3359,
        5679,
        5734,
        5715,
        2968,
        4695,
        3784,
        3499,
        5775,
        5777,
        5779,
        5773,
        5772,
        5769,
        5781,
        4234,
        4423,
        3036,
        2669,
        2058,
        1262,
        2086,
        3729,
        2380,
        77,
        4564,
        5270,
        1373,
        5701,
        5737,
        3534,
        2988,
        3178,
        1609,
        5654,
        1435,
        5498,
        3320,
        3001,
        4243,
        2522,
        2352,
        752,
        4512,
        5725,
        3984,
        3731,
        4490,
        1229,
        2999,
        2859,
        2876,
        2893,
        1575,
        2892,
        4032,
        3143,
        2517,
        1721,
        1893,
        1728,
        4705,
        3478,
        2300,
        2303,
    ]
][key]

In [ ]:
def print_row(x):
    print(x.name)
    for k in (key, (key, "ordinal")):
        print(k, x[k])

In [ ]:
print_row(
    x[x.longname == "Merritt Neighborhood, Oakland City, California, USA"].iloc[0]
)
print_row(x[x.longname == "North Loop Neighborhood, Austin City, Texas, USA"].iloc[0])

In [ ]:
print_row(
    y_n[y_n.longname == "Merritt Neighborhood, Oakland City, California, USA"].iloc[0]
)
print_row(
    y_n[y_n.longname == "North Loop Neighborhood, Austin City, Texas, USA"].iloc[0]
)

In [ ]:
y_n[np.abs(y_n[key] - 0.7734051846829366) < 0.01]

In [ ]:
a = gpd.read_file(shapefiles["native_subdivisions"].path)

In [ ]:
def classify_name(x):
    if "Reservation" in x or "Home Land" in x or "Trust Land" in x or "Rancheria" in x:
        return "Reservation"
    if :
        return "TSA"
    print(x)

In [ ]:
import pickle
with open(shapefiles["historical_congressional"].path, "rb") as f:
    x = pickle.load(f)

In [ ]:
x["state"].apply(lambda x: current_state(x)).all()

In [ ]:
for x in ce_to_name().values():
    classify_name(x)

In [ ]:
a

In [ ]:
u = create_relationships(shapefiles["cities"], shapefiles["state_house"])

In [ ]:
[x for x in u[1] if "Farmington" in str(x)]

In [ ]:
x = school_district_shapefiles()

In [ ]:
x

In [ ]:
b = gpd.read_file(shapefiles["school_districts"].path[0])

In [ ]:
b[b.NAME.apply(lambda x: "Mountain View E" in x)]

In [ ]:
a[a.shortname.apply(lambda x: "city" in x.lower())]

In [ ]:
y[y.STATE_NAME == "Oklahoma"].sort_values("NAME")

In [ ]:
y[y.AIANNHCE == "0335"]

In [ ]:
shapefiles["cities"].load_file()

In [ ]:
x.groupby("state").count().sort_values("STATEFP20")

In [ ]:
x[x.state == "IL"]